# CLV Migration Modeling

In [87]:
!pip install latexify

ERROR: Could not find a version that satisfies the requirement latexify (from versions: none)
ERROR: No matching distribution found for latexify


In [88]:
# import the required packages
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from statsmodels.discrete.discrete_model import Probit
from sklearn.metrics import mean_absolute_error, mean_squared_error

import math
import latexify
from datetime import datetime, timedelta

We are going to **compute CLV of the donors** we saw during case 1 (i.e. the Churn case).

**Multiple methodologies** exist to tackle CLV modeling. During this course you will see two: **data mining** and **migration modeling**. Data mining was already applied during the churn and acquisition cases. Migration modeling however is new. **Migration modeling** looks at customer behavior as a sequential process. At each time period t a customer is either an active or an inactive customer and this behavior depends upon the behavior in t-1 according to a transition matrix ``P``.

We are going to model CLV for the **next 3 years**, with time steps of 1 year:

In [89]:
@latexify.with_latex
def CLV(n, P, v, r):
    return((n * P * v / (1+r)) + (n * P * P * v / (1+r)**2) + (n * P * P * P * v / (1+r)**3))
CLV

With:
- ``n``: initial vector
- ``P``: transition matrix
- ``v``: value vector 
- ``r``: discount rate

Which means we need to have ``n``, ``P``, ``v`` and ``r``.

We are going to use a **discount rate** (**r**) of 0.05. This value is firm-dependent (see financial calculation courses).

This leaves us with the three other parameters:  

``n``: initial vector  
``P``: transition matrix  
``v``: value vector

In [148]:
# set discount rate
DISCOUNT_RATE = 0.05

# 1. Data Exploration

In [91]:
# import all the datasets
extrel = pd.read_csv("./data/extrel.csv")
extrelty = pd.read_csv("./data/extrelty.csv")
communication = pd.read_csv("./data/communication.csv")
payhistory = pd.read_csv("./data/payhistory.csv")
nameaddr = pd.read_csv("./data/nameaddr.csv")
comclas = pd.read_csv("./data/comclas.csv")
commediu = pd.read_csv("./data/commediu.csv")

In [92]:
# inspect first observation extrel
extrel.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,26414,CT,2008-01-30,NaT
1,26419,FP,2005-02-26,NaT
2,26424,FP,2005-02-26,2009-12-21
3,26430,FP,2005-02-26,2019-01-28
4,26430,CT,2010-03-04,NaT


In [93]:
# inspect dtypes of columns
extrel.dtypes

EXTRELNO       int64
EXRELACTCD    object
EXTRELSTDT    object
EXRELDATEN    object
dtype: object

In [94]:
# inspect shape
extrel.shape

(26689, 4)

In [95]:
# inspect payhistory
payhistory.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38,2006-12-28,9.57,19,X,CO
1,39,2006-12-28,41.32,20,X,CO
2,40,2006-12-28,13.10,20,X,CO
3,54091,2006-12-29,2.02,20,X,CO
4,104480,2007-05-16,0.50,20,D,OK


In [96]:
# inspect shape
payhistory.shape

(1205720, 6)

# 2. Time Window

We have the **same restriction** as during the churn case. We have a **gap** until the start of next year for deployment. We use today's data (23 November) to predict CLV. However, this time we are not interested in 1-year churn behavior but in **3-year CLV**. This means we need the **last full 3 years of our available training data**. This means we do not only need the entire year of 2019, but also 2018 and 2017. This means we are only allowed to use data available up until 23 November 2016 for the computation of our *initital state*.

In [97]:
# define the model building time window
end_independent = datetime.strptime("22/11/2016", "%d/%m/%Y")
start_dependent = datetime.strptime("01/01/2017", "%d/%m/%Y")
end_dependent = datetime.strptime("31/12/2019", "%d/%m/%Y")

# 3. Data Preparation

## 3.1. Customers

We need to take a relevant subset of modelable customers just as we did in the churn case. Therefore we have to make **some constraints** in this CLV-case:

1. The donors have to be **active during the independent period**: they needed to start their relationship with the company before the end of the independent period.
2. We are **not interested** in donors that ended their relation before the start of the dependent.
3. Extra condition in comparison to churn case : take only the donors that **made already a transaction by the end of the independent period**. For instance, active donors that have their first payment during the gap will otherwise be missclassified as initially inactive donors which would hinder model performance. + We are interested in value of donors, non-paying donors have (direct monetary) value of zero anyway.

In [98]:
# convert start date EXTRELSTDT and end date EXRELDATEN to datetimes
extrel["EXTRELSTDT"] = pd.to_datetime(extrel["EXTRELSTDT"], format='%Y-%m-%d')
extrel["EXRELDATEN"] = pd.to_datetime(extrel["EXRELDATEN"], format='%Y-%m-%d')

In [99]:
# extract the active donors
active_customers = extrel[(extrel["EXTRELSTDT"] <= end_independent) & 
                        ((extrel["EXRELDATEN"] > start_dependent) | (extrel["EXRELDATEN"].isnull()))]

In [100]:
# get unique customers
active_customers = active_customers["EXTRELNO"].unique()
# convert to DataFrame 
active_customers = pd.DataFrame(active_customers)
# rename column
active_customers.columns = ["EXTRELNO"]
# sort by EXTRELNO 
active_customers = active_customers.sort_values(by=["EXTRELNO"])

In [101]:
# get shape of active_customers dataframe
active_customers.shape

(13552, 1)

In [102]:
# convert payhistory PDATE to datetime
payhistory["PDATE"] = pd.to_datetime(payhistory["PDATE"], format='%Y-%m-%d')

In [103]:
# extract only those transactions which were made before the end of the independent period
payhistory_ind = payhistory[payhistory["PDATE"] <= end_independent]
# extract only observations with positive transactions
payhistory_ind = payhistory_ind[payhistory_ind["PAMT"] > 0]

In [104]:
# sort data by extrelno and pdate
payhistory_ind = payhistory_ind.sort_values(by=["EXTRELNO", "PDATE"])

In [105]:
# get unique set of customers with transactions during the independent period
transaction_customers = payhistory_ind["EXTRELNO"].unique()
# convert to DataFrame
transaction_customers = pd.DataFrame(transaction_customers)
# rename column
transaction_customers.columns = ["EXTRELNO"]

In [106]:
# get shape of transaction_customers dataframe
transaction_customers.shape

(14835, 1)

In [107]:
# merge both tables to get active customers that made transaction during independent period
customers = pd.merge(active_customers, transaction_customers, on="EXTRELNO", how="inner")

In [108]:
# get shape of customers dataframe
customers.shape

(7039, 1)

In [109]:
# check output
customers.head()

,EXTRELNO
0,26414
1,26419
2,26430
3,26431
4,26443


## 3.2. Initial states

Now we can start modeling the still needed values. 

We will start with modeling the **value vector** (``v``) and **initial state vector** (``n``) for all customers, as you will do in the future (**purpose model**). But do note that for this **model building phase** you essentially only need them for your test set as the **transition probability matrix** (``P``) is the only thing you will be fitting on the training set and evaluating on the test set.

We start with the initial state. During ``time period 0``, did the donor actually donate?

Since we are looking at **time periods of 1 year**, this corresponds to recency < 1 year (see churn case).

Luckily, we already created the ``payhistory_ind`` dataset with only information up until the of the **independent period**.

### 3.2.1. Recency

In [110]:
# get the last transactions
recency = payhistory_ind.groupby("EXTRELNO").tail(1)[["EXTRELNO", "PDATE"]]
# get the recency in years
recency["recency"] = recency["PDATE"].apply(lambda x: (end_independent - x).days / 365.)
# drop PDATE
recency = recency.drop("PDATE", axis=1)

In [111]:
# check
recency.head()

,EXTRELNO,recency
0,19,9.909589
120,20,0.035616
275,35,0.035616
397,42,2.871233
627,43,0.369863


### 3.2.2. Initial state


Next, we are going to calculate the ``recency state`` variable, which will be set at ``1`` if the recency is smaller than 1,
and at ``0`` if the recency is larger or equal than 1.

In [112]:
# create recency state variable
recency["initial_state"] = np.where(recency["recency"]>1,0,1)

In [113]:
# check
recency.head()

,EXTRELNO,recency,initial_state
0,19,9.909589,0
120,20,0.035616,1
275,35,0.035616,1
397,42,2.871233,0
627,43,0.369863,1


In [114]:
customers

,EXTRELNO
0,26414
1,26419
2,26430
3,26431
4,26443
...,...
7034,232369
7035,232377
7036,232395
7037,232434


In [115]:
# left outer join with customers dataset
initial_state = pd.merge(left=customers, right=recency, on="EXTRELNO", how="left")

In [116]:
# check shape of initial_state dataframe
initial_state.shape

(7039, 3)

In [117]:
# check output
initial_state.head()

,EXTRELNO,recency,initial_state
0,26414,0.035616,1
1,26419,0.153425,1
2,26430,0.035616,1
3,26431,0.556164,1
4,26443,0.035616,1


In [118]:
# create n
n = initial_state['initial_state'].value_counts()

In [119]:
# check
n

1    6604
0     435
Name: initial_state, dtype: int64

## 3.3. Value Vector

The next thing we are going to compute is the ``value vector``. We **assume this value being equal for each year** (without discounting). Of course, this is not exactly correct. But this actually is **quite robust** and allows for the use of **Markov Chains** (see formula at the top of the notebook).

Since we assume the value vector being equal each year, we are going to **use the donated amount during year 0** as this information will also be **available during the purpose model phase**.

In [120]:
# get payhistory information from dependent period
payhistory_y0 = payhistory[(payhistory["PDATE"] > (end_independent - timedelta(days=365))) & 
                            (payhistory["PDATE"] <= end_independent)]

In [121]:
# define function to get discounted transaction
def get_discounted_transaction(trans_date, trans_val):
    
    # get discounted transaction
    disc_trans = trans_val / ((1 + DISCOUNT_RATE) ** ((trans_date - start_dependent).days / 365.))
    
    # return
    return(disc_trans)
# discount each transaction by a daily discount rate


In [122]:
# discount each transaction by a daily discount rate
payhistory_y0["PAMT_discounted"] = payhistory_y0.apply(lambda x: get_discounted_transaction(x["PDATE"], x["PAMT"]), axis=1)

C:\Users\stefd\AppData\Local\Temp\ipykernel_33864\3071899224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payhistory_y0["PAMT_discounted"] = payhistory_y0.apply(lambda x: get_discounted_transaction(x["PDATE"], x["PAMT"]), axis=1)


In [123]:
# check
payhistory_y0.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS,PAMT_discounted
109,3550005,2015-12-08,23.0,20,D,OK,24.230839
110,3583134,2016-01-11,23.0,20,D,OK,24.120964
111,3615556,2016-02-08,23.0,20,D,OK,24.030853
112,3647372,2016-03-08,23.0,20,D,OK,23.937878
113,3678919,2016-04-06,23.0,20,D,OK,23.845263


In [124]:
# get total of discounted transaction per customer
value_vector = payhistory_y0.groupby("EXTRELNO").agg({"PAMT_discounted": sum}).reset_index()
# rename columns
value_vector.columns = ["EXTRELNO", "CLV"]

In [125]:
# left outer join with customers dataframe
value_vector = pd.merge(customers, value_vector, on="EXTRELNO", how="left")

In [126]:
# get shape of value_vector dataframe
value_vector.shape

(7039, 2)

In [127]:
# check
value_vector.head()

,EXTRELNO,CLV
0,26414,284.319331
1,26419,284.748068
2,26430,284.319331
3,26431,261.352928
4,26443,260.670873


In [128]:
# retrieve minimum value
min(value_vector['CLV'])

-214.26339745266716

This is the individual value per donor, but we want the average value per state.

In [129]:
# inner join value_vector and initial_state
df = pd.merge(value_vector, initial_state, on = 'EXTRELNO', how='inner')

# impute missing values CLV with 0
df['CLV'] = np.where(df['CLV'].isnull(), 0, df['CLV'])

In [130]:
# check
df.head()

,EXTRELNO,CLV,recency,initial_state
0,26414,284.319331,0.035616,1
1,26419,284.748068,0.153425,1
2,26430,284.319331,0.035616,1
3,26431,261.352928,0.556164,1
4,26443,260.670873,0.035616,1


In [131]:
# get mean of each initial_state
v = df.groupby('initial_state')['CLV'].mean()
v

initial_state
0      0.000000
1    287.670114
Name: CLV, dtype: float64

In [132]:
# multiply
n * v

0    0.000000e+00
1    1.899773e+06
dtype: float64

1.899733e+06 corresponds to the current customer value during 1 year (**y0**). 

Both ``n`` and ``v`` will be **recalculated** in our purpose model since this information will also be available then. What won't be available is the **transition matrix** ``P``. We will now calculate it and in a real-life case we would deploy this ``P`` on the new information about initial states and value vectors.

Since we are going to evaluate our model, we will recalculate this values as well: solely for the test set this time.

## 3.4. Transition matrix

We assume the transition matrix to be equal across years, which means we **only need to calculate it once** as well. We will compute the transitions between states between ``y0`` and ``y1`` and **assume** these transition probabilities to be equal across years.

In [133]:
# time window year 1 (y1)
start_y1 = datetime.strptime("01/01/2017", "%d/%m/%Y")
end_y1 = datetime.strptime("31/12/2017", "%d/%m/%Y")

# subset
payments_y1 = payhistory[(payhistory["PDATE"] >= start_y1) & (payhistory["PDATE"] <= end_y1)]

# show
payments_y1.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
122,3958746,2017-01-10,23.0,20,D,OK
123,3989899,2017-02-08,23.0,20,D,OK
124,4020717,2017-03-08,23.0,20,D,OK
125,4051630,2017-04-10,23.0,20,D,OK
126,4081980,2017-05-08,23.0,20,D,OK


In [134]:
# get unique donors of y1
active_y1 = pd.DataFrame(payments_y1['EXTRELNO'].unique())
active_y1.columns = ['EXTRELNO']
active_y1['active_y1'] = int(1)
active_y1.head()
# if donors are in this dataset, they are active, if not: inactive ==> merge with df and fill missings as inactives

,EXTRELNO,active_y1
0,20,1
1,35,1
2,72,1
3,81,1
4,89,1


In [135]:
# merge with df
merge = pd.merge(df, active_y1, on = 'EXTRELNO', how = 'left')

In [136]:
# show
merge.head(20)

,EXTRELNO,CLV,recency,initial_state,active_y1
0,26414,284.319331,0.035616,1,1.0
1,26419,284.748068,0.153425,1,1.0
2,26430,284.319331,0.035616,1,1.0
3,26431,261.352928,0.556164,1,1.0
4,26443,260.670873,0.035616,1,1.0
5,26444,284.319331,0.035616,1,1.0
6,26446,284.292267,0.115068,1,1.0
7,26447,284.319331,0.035616,1,1.0
8,26467,284.319331,0.035616,1,1.0
9,26470,284.319331,0.035616,1,1.0


In [137]:
# impute missing values
merge['active_y1'] = np.where(merge['active_y1'].isnull(), 0, 1)

In [138]:
# show
merge.head(20)

,EXTRELNO,CLV,recency,initial_state,active_y1
0,26414,284.319331,0.035616,1,1
1,26419,284.748068,0.153425,1,1
2,26430,284.319331,0.035616,1,1
3,26431,261.352928,0.556164,1,1
4,26443,260.670873,0.035616,1,1
5,26444,284.319331,0.035616,1,1
6,26446,284.292267,0.115068,1,1
7,26447,284.319331,0.035616,1,1
8,26467,284.319331,0.035616,1,1
9,26470,284.319331,0.035616,1,1


We are now able to **calculate the transition matrix** (``P``). However, we will do this **solely on a training set** and **evaluate** against a test set.

In [139]:
# split into training and test set (70/30)
train, test = train_test_split(merge, test_size=0.3, random_state=33)

In [140]:
# check transition probabilities
P = pd.crosstab(train["initial_state"], train["active_y1"], normalize="index")
P

active_y1,0,1
initial_state,,
0,0.967949,0.032051
1,0.025135,0.974865


## 4. Deployment & Evaluation

## 4.1. Estimation

We will now **deploy this matrix on our test** set to get the predicted value of our customers during the next 3 years. Which we will **compare** against the actual spending of our test sample. To do so, you have to **compute** ``n`` and ``v`` of the test set, which should be easy for you with the code above. Hint: you can use the information which is already available in the test dataframe.

In [146]:
# compute n_test
n_test = test['initial_state'].value_counts()


In [147]:
# compute v_test
v_test = test.groupby('initial_state')['CLV'].mean()


Now compute the **estimated CLV** according to the formula:

In [143]:
@latexify.with_latex
def CLV(n, P, v, r):
    return((n * P * v / (1+r)) + (n * P * P * v / (1+r)**2) + (n * P * P * P * v / (1+r)**3))
CLV

In [149]:
# Write your code here
CLV(n_test, P, v_test, DISCOUNT_RATE)


,0,1
initial_state,,
0,0.0,1.790690e+04
1,0.0,1.473473e+06


## 4.2. Evaluation

Now compute the **actual (discounted) spending** of the test set and **compare** the two.

Exercise:

1. Create a **new dataframe** (called *test_donors*) with all the unique donors from the **test set** and apply our **time window** on it.
2. **Discount** every transactions accordingly and **calculate** the total CLV per donor and merge with *test_donors*.
3. **Impute** missing values with 0.
4. Calculate the **total CLV** of all customers together.
5. **Compare** estimations with true values.

In [150]:
# Write your code here
test_donors = test["EXTRELNO"].unique()
test_donors


array([193017, 174534,  27259, ..., 224771, 221225,  30998], dtype=int64)